# Introduction to Prompting

"Prompting" is the way in which humans communicate with an AI using human language to instruct the LLM systems. 

Prompt engineering is a discipline that creates and optimizes prompts to leverage language models. A prompt engineer translates an idea from conversational language to more precised and optimized instructions to an AI.

In this Jupyter Notebook we provide a complete guide to prompting.

# Table of Contents
- Introduction
- Creating your first prompt
- Prompting Techniques
    - Zero Shot Prompting
    - In-Context Learning and Few-Shot Prompting
        - In-Context Learning
        - Few-Shot Prompting
    - Role Prompting
    - Chain Prompting
    - Chain of Thought Prompting
- Bad Prompting Practices

# Introduction

For a new project we import OpenAI from openai. This will automatically defaults to getting the key using **<u>os.environ.get("OPENAI_API_KEY")</u>**, it's important that for this to automatically work your key must be named OPENAI_API_KEY. 

**Note:** If the code doesn't work, view Readme.md on how to set up this project.

In [2]:
from openai import OpenAI

client = OpenAI()

# Creating your first prompt

In this example we have a few lines of code:
- prompt_system: we explain the system what role we expect it to take
- prompt: we give instructions on what we want to do
- response: we connect to OpenAIs API and give the model instructions

### Example: Story teller

In [8]:
prompt_system = "You are a helpful assistant whose goal is to help write stories."
prompt = """Continue the following story. Write no more than 50 words.

Once upon a time, in a world where animals could speak, a courageous mouse named
Benjamin decided to"""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt}
        ])

print(response.choices[0].message.content)

embark on a dangerous journey to find the legendary Cheese Mountain. With his tiny walking stick in hand, he faced countless challenges, including cunning cats and treacherous traps. But Benjamin pressed on, determined to uncover the secret of the elusive mountain.


# Prompting Techniques

## Zero Shot Prompting

**Zero Shot Prompting** is when a model is asked to produce output with our examples demonstrating the task. We tested it in the previous example. Here is another example.

This example has a variable ***{topic}***. This would be an input from a program that someone can add of a poem writer assistant. We will add the variable at the beggining of the code and you can edit it from there. In future notebooks we will learn how to create our LLM.

In [9]:
topic = "soul"
# This is the prompt, we use the topic variable that will be our input somewhere where we implement our model
#
prompt_system = "You are a helpful assistant whose goal is to write short poems."
prompt = f"""Write a short poem about {topic}."""

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt}
        ])

print(response.choices[0].message.content)

In the depths of the heart, a spark ignites,  
A precious essence, a guiding light,  
The soul that dances, pure and true,  
Connecting us to all that's new.  
So cherish your soul, let it shine bright,  
For it's a beacon in the darkest night.


## In-Context Learning and Few-Shot Prompting

**In context learning** is an approach where a model learns from examples or demonstrations in the prompt.

**Few-Shot prompting** is a subset of in context learning that presents the model with a small set of relevant examples. This is useful for more complex tasks.

Unlike zero shot, we use examples to improve performance.

### In-Context learning

As you can see in this example we provide three examples with three verses, if you see the output it follows the same structure, guiding itself from examples.

In [11]:
input_topic = "love"

prompt_system = "You are a helpful assistant whose goal is to write short poems."
prompt = f"""Write a short poem about {topic}."""
examples = {"nature": """Birdsong fills the air,\nMountains high and valleys deep,\nNature's music sweet.""", 
            "winter": """Snow blankets the ground,\nSilence is the only sound,\nWinter's beauty found."""
            }

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_system},
        {"role": "user", "content": prompt.format(topic="nature")},
        {"role": "assistant", "content": examples["nature"]},
        {"role": "user", "content": prompt.format(topic="winter")},
        {"role": "assistant", "content": examples["winter"]},
        {"role": "user", "content": prompt.format(topic=input_topic)}
        ])

print(f"Output:\n\n{response.choices[0].message.content}\n")

Output:

In the heart's quiet space,
The soul finds its resting place,
Guiding with gentle grace.



### Few-Shot prompting

For this example that is more complex we will use LangChain, this is a library designed to facilitate use of LLMs. We will go further into the topic in a future notebook.

You can see the printed prompt to get an idea of how it works by putting together all this code into a final prompt.

**Note:** While few-shot learning is effective, it encounters challenges, mainly when tasks are complex. More advanced strategies, like chain-of-thought prompting, become increasingly valuable in such cases. This technique breaks down complex problems into simpler phases, offering examples for each stage and enhancing the model’s logical reasoning capacity.

In [13]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {"color": "red", "emotion": "passion"},
    {"color": "blue", "emotion": "serenity"},
    {"color": "green", "emotion": "tranquility"},
]

example_formatter_template = """
Color: {color}
Emotion: {emotion}
"""

example_prompt = PromptTemplate(
    input_variables=["color", "emotion"],
    template=example_formatter_template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""Here are some examples of colors and the emotions associated with \
them:""",
    suffix="""Now, given a new color, identify the emotion associated with \
it:\nColor: {input}\nEmotion:""",
    input_variables=["input"],
    example_separator="\n",
)

formatted_prompt = few_shot_prompt.format(input="purple")

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=PromptTemplate(template=formatted_prompt,
input_variables=[]))

# Run the LLMChain to get the AI-generated emotion associated with the input
# color
response = chain.run({})

# We print the example of the prompt sent
print("Input: 'Color: purple'")
print("Output: 'Emotion:", f"{response}'")
print(f"\nPrompt:\n\n{chain.prompt.template}")

Input: 'Color: purple'
Output: 'Emotion: royalty'

Prompt:

Here are some examples of colors and the emotions associated with them:

Color: red
Emotion: passion


Color: blue
Emotion: serenity


Color: green
Emotion: tranquility

Now, given a new color, identify the emotion associated with it:
Color: purple
Emotion:


## Role Prompting

Role prompting involves instructing the LLM to assume a specific role or identity for task execution, such as functioning as a copywriter. This instruction can influence the model’s response by providing context or perspective for the task. When working with role prompts, the iterative process includes:

1. Defining the role in the prompt. For example, “As a copywriter, create engaging catchphrases for AWS services.”
2. Utilizing the prompt to generate a response from an LLM.
3. Evaluating the response and refining the prompt as needed for improved outcomes".

In [17]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

template = """
As a futuristic robot band conductor, I need you to help me come up with a song title.
What's a cool song title for a song about {theme} in the year {year}?"""

prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template=template,
    )

# Input data for the prompt
input_data = {"theme": "interstellar travel", "year": "3030"}

# Create LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the LLMChain to get the AI-generated song title
response = chain.run(input_data)

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: "Galactic Odyssey: Journey Through the Stars in 3030"


## Chain Prompting

Chain Prompting involves linking a series of prompts sequentially, where the output from one prompt serves as the input for the next. When implementing chain prompting with LangChain, consider the following steps:

- Identify and extract relevant information from the generated response.
- Develop a new prompt using this extracted information, ensuring it builds upon the previous response.
- Continue this process as necessary to reach the intended result.

PromptTemplate class is designed to simplify the creation of prompts with dynamic inputs. This feature is particularly useful in constructing a prompt chain that relies on responses from previous prompts.

In [19]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Prompt 1
template_question = """
What is the name of the famous scientist who developed the theory of general relativity?
Answer: """

prompt_question = PromptTemplate(template=template_question, input_variables=[])

# Prompt 2
template_fact = """
Provide a brief description of {scientist}'s theory of general relativity.
Answer: """

prompt_fact = PromptTemplate(input_variables=["scientist"], template=template_fact)

# Create the LLMChain for the first prompt
chain_question = LLMChain(llm=llm, prompt=prompt_question)

# Run the LLMChain for the first prompt with an empty dictionary
response_question = chain_question.run({})

# Extract the scientist's name from the response

scientist = response_question.strip()

# Create the LLMChain for the second prompt
chain_fact = LLMChain(llm=llm, prompt=prompt_fact)

# Input data for the second prompt
input_data = {"scientist": scientist}

# Run the LLMChain for the second prompt
response_fact = chain_fact.run(input_data)

print("Scientist:", scientist)
print("Fact:", response_fact)


Scientist: Albert Einstein
Fact: Albert Einstein's theory of general relativity, published in 1915, describes gravity as a curvature in the fabric of spacetime caused by the presence of mass and energy. According to this theory, massive objects like planets and stars warp the fabric of spacetime, causing other objects to move along curved paths. General relativity also predicts phenomena such as gravitational time dilation, gravitational waves, and the bending of light around massive objects.


## Chain of Thought Prompting

Chain of Thought Prompting (CoT) is a method designed to prompt Large Language Models to articulate their thought process, enhancing the accuracy of the results. This technique involves presenting examples that showcase the reasoning process, guiding the LLM to explain its logic while responding to prompts. CoT has proven beneficial for arithmetic, common-sense reasoning, and symbolic thinking tasks.

In the context of LangChain, CoT offers several advantages. Firstly, it simplifies complex tasks by enabling the LLM to break down challenging problems into more manageable steps. This feature is valuable for tasks requiring calculations, logical analysis, or multi-step reasoning. Secondly, CoT can guide the model through a series of related prompts, fostering more coherent and contextually appropriate outputs. This can result in more precise and practical responses, especially in tasks requiring a thorough understanding of the problem or subject matter.

However, there are limitations to CoT that should be considered. One limitation is that it is effective primarily with models with around 100 billion parameters or more. Smaller models often produce nonsensical thought processes, reducing accuracy compared to traditional prompting methods. Another limitation is that CoT’s effectiveness varies across different types of tasks. While it shows significant benefits for tasks involving arithmetic, common sense, and symbolic reasoning, its impact on other tasks might be less meaningful.

# Tips for Effective Prompt Engineering

Prompt engineering is an iterative process, often requiring multiple adjustments to obtain the most accurate response. As LLMs continue integrating into various products and services, proficiency in devising effective prompts will become crucial. Here are the general rules to follow:

- Be specific with your prompt: Include sufficient context and detail to guide the LLM toward the intended output.
- Force conciseness when necessary.
- Encourage the model to describe why it is the way it is: This can result in more precise solutions, particularly for complex tasks.